# Removing Duplicate Students From Foot Traffic

In February 2019, I created an electronic sign-in sheet for the academic advising office foot traffic and launched it. For decades, the signin sheet was a paper form that recorded all the students who walked into the office, what they came in for, and the type of student they were. When I started data analysis, I transfered tens of thousands of students from the paper forms into electronic form in Excel. This was the only way to have the data be useful. Prior to my initiative, the data had never been aggregated by the college. It was recorded but not used to its fullest potential. The electronic signin sheet, obviously made my analysis much easier.

A separate algorithm sets the data up to be analyized. That is the csv that is referenced in the beginning--"Dashboard Setup.csv." One of the things the Director of Advising wanted to track was the workload of each advisor. As I dug into the data, it became clear very quickly that advisors had different entry patterns. Some would enter a student every time they interacted with them. For instance, an advisor could enter a students a half a dozen or a dozen times in a single week if they had multiple emails with the student. To address this, I wrote this algorithm to filter out duplicate students so that we could more accurately assess the number of students seen by each advisor. 

In [ ]:
import pandas as pd
import numpy as np
import re
import sklearn

In [ ]:
#Import raw data that has been processed in 'Advising Foot Traffic' 
#notebook--'Dashboard Setup.csv'

adv_raw = pd.read_csv('Dashboard Setup.csv')

adv_raw['DATE'] = pd.to_datetime(adv_raw['DATE'])

In [ ]:
#extract week

adv_raw['WEEK'] = adv_raw['DATE'].dt.isocalendar().week

In [ ]:
# Calculate the number of dupilcate names in the foot traffic data.
len(adv_raw) - len(adv_raw.drop_duplicates(['MONTH', 'NAME', 'ADVISOR SIGN']))

In [ ]:
#This dataframe shows the counts of duplicates per month by advisor without
#any additional filtering
(pd.DataFrame(adv_raw[adv_raw.duplicated(['MONTH', 'NAME', 'ADVISOR SIGN'])]
    .groupby('ADVISOR SIGN')['ID'].count()).rename(columns = {'ID':'COUNT'})
    .reset_index().sort_values('COUNT', ascending = False)
    .reset_index(drop = True))
)

In [ ]:
#This dataframe we take into account the 'distance' column. We isolate just 
#the instances in which there are no null values in the row. Null values
#are either because there were appointments or because they were walkins.
#This leaves us with a dataframe of 14705 interactions.

#Out of that 14705, 2326 were duplicates. So these duplicates reflect
#instances in which the advisor chose to keep entering the student when
#they called or emailed them. Notice that some of your lower people who
#had duplicates (those <= 20), drop completely off or down to low single 
#digits. 

mask = adv_raw['DISTANCE'].isnull() == False

df2 = adv_raw[mask]

adv_dups = (pd.DataFrame(df2[df2.duplicated(['MONTH', 'NAME', 'ADVISOR SIGN'])]
             .groupby('ADVISOR SIGN')['ID'].count()).rename(columns = {'ID':'COUNT'})
             .reset_index().sort_values('COUNT', ascending = False)
             .reset_index(drop = True)
           )

adv_dups

In [ ]:
#Below is a dataframe with all students an advisor assisted throughout a 
#year REGARDLESS OF DUPLICATION. 
all_adv = (pd.DataFrame(adv_raw.groupby('ADVISOR SIGN')['ID'].count())
            .reset_index().rename(columns = {'ID':'COUNT'})
            .sort_values('COUNT', ascending = False)
            .reset_index(drop = True)
          )

all_adv

In [ ]:
#Now we merge all of the signatures with the duplicated signatures
all_and_dups = (all_adv.merge(adv_dups, how = 'left', on = 'ADVISOR SIGN')
                       .rename(columns = {'COUNT_y':'DUPLICATED',
                                         'COUNT_x':'FULL_COUNT'})
                       .fillna(0)
               )

#clean up the dataframe to eliminate ['??', 'LEFT', '8:03']
all_and_dups = all_and_dups[all_and_dups['ADVISOR SIGN'].isin(['??', 'LEFT', '8:03']) == False]

#Calculate percent of the signatures from each advisor that are duplicated
#students
all_and_dups['PERC_OF_STUDENTS'] = round((all_and_dups['DUPLICATED']/all_and_dups['FULL_COUNT']) * 100, 3)

#Sort from highest to lowest of percent of students that are duplicated 
#by each advisor
all_and_dups = (all_and_dups.sort_values('PERC_OF_STUDENTS', ascending = False).reset_index(drop = True)
                            .reset_index().rename(columns = {'index':'PERC_RANK'})
                            .sort_values('FULL_COUNT', ascending = False).reset_index(drop = True)
               )
all_and_dups

In [ ]:
#Only run through this cell once. Each time you run it, it will 
#add another to the ranking
all_and_dups['PERC_RANK'] = all_and_dups['PERC_RANK'] + 1

In [ ]:
all_and_dups = all_and_dups.reset_index().rename(columns = {'index':'RANK'})

all_and_dups['RANK'] = all_and_dups['RANK'] + 1

One of the revelations of this analysis is that the advisor who assisted the second highest number of students, when considering hte raw count, had 15% of the students entered as duplicates, which draws that number way down. The second consideration is that advisors who oversee the advising email account will, by default, have far more students in their count than advisors who do not have access to that account. 

This analysis influenced the Director of Advising to put every advisor in the college in the advising email account because the workload was so drastically imbalanced. 

In [ ]:
all_and_dups